# Delphi Demo - Causal Analysis Graphs from <span style='color:royalblue; font-style: italic'>Software</span>

This is a Jupyter notebook created to showcase the program analysis-specific design and capabilities of
the Delphi package, available at [https://github.com/ml4ai/delphi](https://github.com/ml4ai/delphi). The program analysis capabilities of Delphi are being developed as part of the AutoMATES program.  

If you are viewing this as a prerendered HTML document, it was generated with the version of Delphi corresponding to the commit hash below.

In [ ]:
!git rev-parse HEAD

In [ ]:
%load_ext autoreload
%autoreload 2

## Original Fortran program

Below is a small FORTRAN program that calculates crop yield given rainfall. Executing the cell below writes the FORTRAN program to a file called `crop_yield.f`

In [ ]:
%%writefile crop_yield.f
************************************************************************
*     UPDATE_EST - Updates the estimated yield of magic beans given 
*       some additional amount of rainfall
************************************************************************
*
*     VARIABLES
*     
*     INPUT RAIN      = Additional rainfall
*
*     INPUT YIELD_EST = Crop yield to update
*
************************************************************************
      SUBROUTINE UPDATE_EST(RAIN, TOTAL_RAIN, YIELD_EST)
        DOUBLE PRECISION RAIN, YIELD_EST, TOTAL_RAIN
        TOTAL_RAIN = TOTAL_RAIN + RAIN

*       Yield increases up to a point
        IF(TOTAL_RAIN .le. 40) THEN
            YIELD_EST = -(TOTAL_RAIN - 40) ** 2 / 16 + 100

*       Then sharply declines
        ELSE
            YIELD_EST = -TOTAL_RAIN + 140
        ENDIF

      END SUBROUTINE UPDATE_EST

************************************************************************
*     CROP_YIELD - Estimate the yield of magic beans given a simple 
*       model for rainfall
************************************************************************
*
*     VARIABLES
*     
*     INPUT MAX_RAIN   = The maximum rain for the month
*     INPUT CONSISTENCY = The consistency of the rainfall 
*       (higher = more consistent)
*     INPUT ABSORPTION = Estimates the % of rainfall absorbed into the
*       soil (i.e. % lost due to evaporation, runoff)
*
*     OUTPUT YIELD_EST = The estimated yield of magic beans
*
*     DAY              = The current day of the month
*     RAIN             = The rainfall estimate for the current day
*
************************************************************************
      PROGRAM CROP_YIELD
      IMPLICIT NONE

      INTEGER DAY
      DOUBLE PRECISION RAIN, YIELD_EST, TOTAL_RAIN
      DOUBLE PRECISION MAX_RAIN, CONSISTENCY, ABSORPTION

      MAX_RAIN = 4.0
      CONSISTENCY = 64.0
      ABSORPTION = 0.6
      
      YIELD_EST = 0
      TOTAL_RAIN = 0
 
      DO 20 DAY=1,31
*       Compute rainfall for the current day
        RAIN = (-(DAY - 16) ** 2 / CONSISTENCY + MAX_RAIN) * ABSORPTION

*       Update rainfall estimate
        CALL UPDATE_EST(RAIN, TOTAL_RAIN, YIELD_EST)
        PRINT *, "Day ", DAY, " Estimate: ", YIELD_EST

   20 ENDDO

      PRINT *, "Crop Yield(%): ", YIELD_EST

      END PROGRAM CROP_YIELD

In [ ]:
!pwd

In [ ]:
import os
os.environ["CLASSPATH"] = (
    "../delphi/program_analysis/autoTranslate/bin/*"
)
original_fortran_file = "crop_yield.f"
preprocessed_fortran_file = "crop_yield_preprocessed.f"

In [ ]:
import ast
from delphi.program_analysis.autoTranslate.scripts import (
    f2py_pp,
    translate,
    get_comments,
    pyTranslate,
    genPGM,
)
import subprocess as sp
import xml.etree.ElementTree as ET
import json
f2py_pp.process(original_fortran_file, preprocessed_fortran_file)
xml_string = sp.run(
    [
        "java",
        "fortran.ofp.FrontEnd",
        "--class",
        "fortran.ofp.XMLPrinter",
        "--verbosity",
        "0",
        "crop_yield_preprocessed.f",
    ],
    stdout=sp.PIPE,
).stdout

trees = [ET.fromstring(xml_string)]
comments = get_comments.get_comments(preprocessed_fortran_file)
outputDict = translate.analyze(trees, comments)
pySrc = pyTranslate.create_python_string(outputDict)
asts = [ast.parse(pySrc)]
pgm_dict = genPGM.create_pgm_dict(
    "crop_yield_lambdas.py", asts, "crop_yield.json"
)
with open("crop_yield.json", "w") as f:
    f.write(json.dumps(pgm_dict, indent=2))

## Extracted lambda functions

In [ ]:
import delphi.jupyter_tools as jt
jt.display("crop_yield_lambdas.py")

## JSON-Serialized GrFN

In [ ]:
jt.display("crop_yield.json")

## Executable DBN - Loop plate representation

In [ ]:
from delphi.program_analysis.scopes import Scope
root = Scope.from_json("crop_yield.json")
A = root.to_agraph()
jt.display_image(A.draw(format='png', prog='dot'))

## High-level representation of CAG from program

In [ ]:
import crop_yield_lambdas
from delphi.program_analysis.ProgramAnalysisGraph import ProgramAnalysisGraph
G = ProgramAnalysisGraph.from_agraph(A, crop_yield_lambdas)
G.initialize()

In [ ]:
from delphi.visualization import visualize
visualize(G, show_values = True)

In [ ]:
G.update()
visualize(G, show_values = True)

In [ ]:
G.update()
visualize(G, show_values = True)

## Sensitivity Analysis

In [ ]:
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
import numpy as np
from matplotlib import pyplot as plt
from delphi.utils import compose, rcompose
from delphi.program_analysis.ProgramAnalysisGraph import ProgramAnalysisGraph

def make_plots(n_samples, deterministic = True):
    variables = ('RAIN', 'TOTAL_RAIN', 'YIELD_EST')
    vals = {k:[] for k in variables}
    days = {k:[] for k in variables}
    palette = sns.color_palette()
    colors = {k:palette[i] for i, k in enumerate(vals)}
    fig, axes = plt.subplots(1,len(vals), figsize=(18, 5))
    ax = {k:axes[i] for i, k in enumerate(vals)}

    for _ in range(n_samples):
        G = ProgramAnalysisGraph.from_agraph(A, crop_yield_lambdas)
        if not deterministic:
            G.nodes['MAX_RAIN']['init_fn'] = lambda: np.random.normal(4, 1)
        G.initialize()
        for i in range(1,31):
            G.update()
            for k in vals:
                vals[k].append(G.nodes[k]['value'])
                days[k].append(G.nodes['DAY']['value'])

    for k in vals:
        sns.lineplot(days[k], vals[k], ax = ax[k], label=k, color=colors[k])
        ax[k].set_xlabel('DAY', fontsize=20)
        ax[k].set_ylabel(k, fontsize=20)

    plt.tight_layout()

make_plots(10, deterministic=False)